# Descarga y procesamiento de datos de la Secretaría de Salud

## Descripción
Script para descargar y procesar PDFs del [Coronavirus-Comunicado Técnico Diario](https://www.gob.mx/salud/documentos/coronavirus-covid-19-comunicado-tecnico-diario-238449) de la **Secretaría de Salud** del Gobierno de México.

La descarga se realiza y se archiva en el directorio `/ssalud_pdf`

El procesamiento consiste en convertir los PDFs en tablas en formato `tsv` y añadir anotaciones sobre casos nuevos, que en el PDF están indicadas por las filas azules. Los datos puros se encuentran en `datos/tablas_originales`

## Descarga de PDFs

### Código

In [15]:
import requests
from bs4 import BeautifulSoup

In [3]:
def extract_documents(url, outdir='.'):
    '''
    Funciona con el formato en 2020.03.25
    '''
    
    # obtener sitio
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # encotnrar seccion de documentos
    # 2020.03.25: Seccion de documentos bajo <div class="clearfix">
    documents = soup.find('div', attrs={'class': 'clearfix'})
            
    # links de los documentos
    file_urls = []
    
    for href in documents.find_all('a', href=True):
        file_urls.append(href['href'])
        
    # descargar
    for i in range(len(file_urls)):
        file_url = file_urls[i]
        file_name = file_urls[i].split('/')[-1]
        response = requests.get(f'https://www.gob.mx{file_url}')
        
        with open(f'{outdir}/{file_name}', 'wb') as f:
            f.write(response.content)

### Descargas

In [5]:
descargas_dir = '../datos/ssalud_pdf'

#### 2020.03.25

In [ ]:
url = 'https://www.gob.mx/salud/documentos/coronavirus-covid-19-comunicado-tecnico-diario-238449'
extract_documents(url, outdir=descargas_dir)

#### 2020.03.27

In [35]:
url = 'https://www.gob.mx/salud/documentos/coronavirus-covid-19-comunicado-tecnico-diario-238449'
extract_documents(url, outdir=descargas_dir)

#### 2020.03.28

In [6]:
url = 'https://www.gob.mx/salud/documentos/coronavirus-covid-19-comunicado-tecnico-diario-238449'
extract_documents(url, outdir=descargas_dir)

## Procesamiento de PDFs

### Código

In [21]:
import camelot
import pandas as pd
import numpy as np

In [22]:
def parse_pdf(file, paginas='all', casos_nuevos=None):
    '''
    '''
    tables = camelot.read_pdf(file, pages=paginas, flavor='stream', row_tol=10)
    print('{} paginas'.format(tables.n))

    # juntar todas las tablas
    tables_list = []
    for i in range(0,tables.n):
        df_i = tables[i].df
        
        # buscar columnas con nan
        df_i= df_i.replace(r'^\s*$', np.nan, regex=True)
        df_i = df_i.replace('NA', np.nan)
        
        # remove na columns
        df_i = df_i.dropna(axis='columns', thresh=10)
        df_i = df_i.T.reset_index(drop=True).T
        
        # quitar filas de titulo
        # por si proceso una pagina vacia
        if len(df_i.columns) != 0:
            df_i[0] = df_i[0].fillna('notnum')
            df_i = df_i[df_i[0].apply(lambda x: x.isnumeric())]
        
            # argregar a lista
            tables_list.append(df_i)

    # juntar tabla unica
    df = pd.concat(tables_list)
    
    
    # renombrar columnas
    df.columns = ['num_caso', 'estado', 'sexo', 'edad', 'fecha_inicio_sintomas', 'id_rt-pcr', 'procedencia', 'fecha_llegada_mx']
    
    # actualizar index
    df.set_index('num_caso', inplace=True, verify_integrity=True)
    df.index.name = None
    
    # convertir a datetime format
    df['fecha_inicio_sintomas'] = pd.to_datetime(df['fecha_inicio_sintomas'], format='%d/%m/%Y')
    df['fecha_llegada_mx'] = pd.to_datetime(df['fecha_llegada_mx'], format='%d/%m/%Y')
    
    # limpiar nombres de estados
    df['estado'] = df['estado'].str.title()
    df['estado'] = df['estado'].str.replace('Ciudad De México', 'Ciudad de México')
    df['estado'] = df['estado'].str.replace('"Estados \nUnidos"', 'Estados Unidos')
    
    # CASOS NUEVOS
    
    if casos_nuevos:
        # interpretar string de casos nuevos
        # by: https://stackoverflow.com/users/190597/unutbu
        # in: https://stackoverflow.com/questions/4726168/parsing-command-line-input-for-numbers
        result = set()

        for part in casos_nuevos.split(','):
            x = part.split('-')
            result.update(range(int(x[0]), int(x[-1]) + 1))

        lista_casos_nuevos = sorted(result)

        # generar lista con anotaciones de casos nuevos
        lista_nuevos = []

        for i in df.index:
                if int(i) in lista_casos_nuevos:
                    lista_nuevos.append(True)
                else:
                    lista_nuevos.append(False)

        df['casos_nuevos'] = lista_nuevos
    
    else:
        print('No se agregara información sobre casos nuevos. (Celdas en azul.)')

    return df

### Processing

In [23]:
pdf_dir = '../datos/ssalud_pdf/'
save_dir = '../datos/tablas_originales/'

In [24]:
def save_tsv(pdf_name, save_name, celdas_azules):
    
    df = parse_pdf(f'{pdf_dir}{pdf_name}',
                   paginas='all',
                   casos_nuevos=celdas_azules)
    
    df.to_csv(f'{save_dir}{save_name}',
              sep='\t',
              index=True)

#### 2020-03-16

In [25]:
pdf = 'Tabla_casos_positivos_resultado_InDRE_2020.03.16.pdf'
celdas_azules = '49-51, 54-57, 60-82'

save_name = '20200316_positivos.tsv'

save_tsv(pdf, save_name, celdas_azules)

2 paginas


#### 2020-03-17

In [26]:
pdf = 'Tabla_casos_positivos_resultado_InDRE_2020.03.17.pdf'
celdas_azules = '64,66,82,86-93'

save_name = '20200317_positivos.tsv'

save_tsv(pdf, save_name, celdas_azules)

2 paginas


#### 2020-03-18

In [27]:
pdf = 'Tabla_casos_positivos_COVID-19_resultado_InDRE_2020.03.18.pdf'
celdas_azules = '92-101,103-105,107-118'

save_name = '20200318_positivos.tsv'

save_tsv(pdf, save_name, celdas_azules)

3 paginas


#### 2020-03-19

In [28]:
pdf = 'Tabla_casos_positivos_COVID-19_resultado_InDRE_2020.03.19.pdf'
celdas_azules = '92,112-115,117-123,125,130,133-164'

save_name = '20200319_positivos.tsv'

save_tsv(pdf, save_name, celdas_azules)

4 paginas


#### 2020-03-20

In [29]:
pdf = 'Tabla_casos_positivos_COVID-19_resultado_InDRE_2020.03.20.pdf'
celdas_azules = '133,135,138,142,143,145,154-158,160-163,166-168,170,181,182,184-188,190-203'

save_name = '20200320_positivos.tsv'

save_tsv(pdf, save_name, celdas_azules)

4 paginas


#### 2020-03-21

In [30]:
pdf = 'Tabla_casos_positivos_COVID-19_resultado_InDRE_2020.03.21.pdf'
celdas_azules = '199-205,207,208,210,211,213,214,216,218-251'

save_name = '20200321_positivos.tsv'

save_tsv(pdf, save_name, celdas_azules)

5 paginas


#### 2020-03-22

In [31]:
pdf = 'Tabla_casos_positivos_COVID-19_resultado_InDRE_2020.03.22.pdf'
celdas_azules = '237,244,246,248-251,255,257,259,261-264,266,267,270,271,273-286,288-316'

save_name = '20200322_positivos.tsv'

save_tsv(pdf, save_name, celdas_azules)

7 paginas


#### 2020-03-23

In [32]:
pdf = 'Tabla_casos_positivos_COVID-19_resultado_InDRE_2020.03.23.pdf'
celdas_azules = '126,182,219,295-297,307,313-314,322,324-326,330-367'

save_name = '20200323_positivos.tsv'

save_tsv(pdf, save_name, celdas_azules)

7 paginas


#### 2020-03-24

In [33]:
pdf = 'Tabla_casos_positivos_COVID-19_resultado_InDRE_2020.03.24.pdf'
celdas_azules = '333,334,336,339-342,349,361-364,379,381-405'

save_name = '20200324_positivos.tsv'

save_tsv(pdf, save_name, celdas_azules)

8 paginas


#### 2020-03-25

In [34]:
pdf = 'Tabla_casos_positivos_COVID-19_resultado_InDRE_2020.03.25.pdf'
celdas_azules = '126,397-403,407,408,410,411,415,416,418,420,422-475'

save_name = '20200325_positivos.tsv'

save_tsv(pdf, save_name, celdas_azules)

9 paginas


#### 2020-03-26

In [35]:
pdf = 'Tabla_casos_positivos_COVID-19_resultado_InDRE_2020.03.26.pdf'
celdas_azules = '428,443-445,447,448,451,453,454,462,468,470,472,473,476,477,479-481,483-486,490,493-499,501-507,509-520,522,523,526,528-585'

save_name = '20200326_positivos.tsv'

save_tsv(pdf, save_name, celdas_azules)

13 paginas


#### 2020-03-27

In [36]:
pdf = 'Tabla_casos_positivos_COVID-19_resultado_InDRE_2020.03.27.pdf'
celdas_azules = '101-105,110-112,114-115,240,346,351,372,374,553-564,566,568,582,584-586,588,589,592,593,595,597,599,600,602-611,614,617,620,625,628,631,642-717'

save_name = '20200327_positivos.tsv'

save_tsv(pdf, save_name, celdas_azules)

15 paginas


#### 2020-03-28

In [37]:
pdf = 'Tabla_casos_positivos_COVID-19_resultado_InDRE_2020.03.28.pdf'
celdas_azules = '49-56,146,249,251,353,463,664,667,676,678,680,686,692,695,697,703,707,725,726,728-730,732,733,738,740,742,744,748,750,752,755-848'

save_name = '20200328_positivos.tsv'

save_tsv(pdf, save_name, celdas_azules)

16 paginas
